In [1]:
import re 
import os
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
from collections import Counter
from sklearn.model_selection import train_test_split


import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu") # cool M2 chip GPU acceleration

# 1. Access the corresponding .txt, .wav, and .avi files for each EDA label
# Extract the conversation filename and speaker informationfrom the dataset
eda_df = pd.read_csv("eda_iemocap_no_utts_dataset.csv")
eda_df = eda_df[["speaker", "utt_id", "EDA"]]
filename_ids = []
speaker_M_F = []
session_numbers = []
for i, row in eda_df.iterrows():
    match = re.search(r"b'(Ses(\d+)[MF]_.+\d+.*)_([MF])", row["speaker"])
    filename_ids.append(match.group(1))
    session_numbers.append(int(match.group(2))) 
    speaker_M_F.append(match.group(3))
eda_df = eda_df.drop(columns=["speaker"])
eda_df["filename"] = filename_ids
eda_df["filename"] = eda_df["filename"].astype(str)
eda_df["session_number"] = session_numbers
eda_df["session_number"] = eda_df["session_number"].astype(int)
eda_df["speaker"] = speaker_M_F
eda_df["speaker"] = eda_df["speaker"].astype(str)
eda_df["utt_id"] = eda_df["utt_id"].astype(int)
# Access transcipt files based on filename
utt_df = []
root_dir = "IEMOCAP_full_release/"
for i in range(1, 6):
    directory = os.path.join(root_dir, f"Session{i}/dialog/transcriptions/")
    for entry in os.scandir(directory):  
        if entry.is_file() and entry.path.endswith(".txt"):  # check if it's a file
            with open(entry.path, "r") as file:
                filename = entry.path.split("/")[-1][:-4]
                lines = file.readlines()
                for order, line in enumerate(lines):
                    speaker_info, utterance = line.split(":")[0], line.split(":")[1]
                    pattern = r"(F|M)(\d+)\s\[(\d+\.\d+)-(\d+\.\d+)\]"
                    match = re.search(pattern, speaker_info)
                    if match is None:
                        continue
                    speaker_f_m = match.group(1)
                    utt_id = match.group(2)
                    start = match.group(3)
                    end = match.group(4)
                    utt_df.append({"utt_id": int(utt_id), "filename": str(filename), "start": float(start), "end": float(end), "speaker": str(speaker_f_m.strip()), "utterance": utterance.strip(), "session_number": int(i), "original_order": order})
utt_df = pd.DataFrame(utt_df)
# Combine the EDA and utterances together
final_df = pd.merge(eda_df, utt_df, on=["utt_id", "session_number", "filename", "speaker"])
final_df
# TODO: Double check if there is really a perfect merge match

/Users/kaleenshrestha/anaconda3/envs/dac/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,utt_id,EDA,filename,session_number,speaker,start,end,utterance,original_order
0,0,sd,Ses01M_impro07,1,M,2.6812,7.9800,Check this out. You know how I've told you I'...,0
1,0,b,Ses01M_impro07,1,F,7.6300,8.5700,Yeah.,1
2,1,sd,Ses01M_impro07,1,M,8.2200,14.7500,"Well, this is totally random, I got this full ...",2
3,1,qy,Ses01M_impro07,1,F,13.9500,21.1200,[LAUGHTER]. For softball? That's unbelievable....,3
4,2,qy,Ses01M_impro07,1,M,15.5400,20.6700,For softball. They're going to pay me to go t...,4
...,...,...,...,...,...,...,...,...,...
10034,24,qy,Ses05F_script01_3,5,F,404.1923,406.6600,Do you still feel like that?,63
10035,39,sd,Ses05F_script01_3,5,M,407.9600,410.7823,"I I want you now, Annie.",64
10036,25,qy,Ses05F_script01_3,5,F,410.4317,427.7079,Because you can't feel like that anymore Chris...,65
10037,40,sd,Ses05F_script01_3,5,M,426.7965,431.8242,"Oh Annie. Annie, I am going to make you a for...",66


In [2]:
# 2. Create a dataset following agreement estimation project for each modality or each utterance
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


labels = list(set(Counter(final_df["EDA"]).keys())) # there are 34 labels
labels_to_num_mapping = {}#'CONCESSION': 0, 'FACTS': 1, 'INTEREST': 2, 'POSITIVE EXPECTATIONS': 3, 'POWER': 4, 'PROCEDURAL': 5, 'PROPOSAL': 6, 'RESIDUAL': 7, 'RIGHTS': 8}
for i, label in enumerate(labels):
    labels_to_num_mapping[label] = i

class DialogActDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(labels_to_num_mapping[self.labels[idx]])
        return item     

    def __len__(self):
        return len(self.labels)
    
df_train, df_test = train_test_split(final_df, train_size=0.8)
df_test, df_val = train_test_split(df_test, train_size=0.5)

train_texts, train_labels = [utt.lower() for utt in list(df_train["utterance"])], list(df_train["EDA"])
val_texts, val_labels = [utt.lower() for utt in list(df_val["utterance"])], list(df_val["EDA"])
test_texts, test_labels = [utt.lower() for utt in list(df_test["utterance"])], list(df_test["EDA"])

model_card = "FacebookAI/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_card)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

train_dataset = DialogActDataset(train_encodings, train_labels)
val_dataset = DialogActDataset(val_encodings, val_labels)
test_dataset = DialogActDataset(test_encodings, test_labels)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(model_card, num_labels=len(labels))


metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(
    output_dir="./results/roberta-large/",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# # 3. Run some training on 80% of the scripted data on the lab machine
# # 4. Determine the training/test splits (think of emotion distribution, scenarios, speaker)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/kb/f069hzld211df9hzjrzd_8z80000gn/T/ipykernel_13896/4185949331.py:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/Users/kaleenshrestha/anaconda3/envs/dac/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 